# OFFLINE/ONLINE LEARNING

# import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import copy

#### define parameters of the study

In [2]:
study_number = 3 # choose from 1, 2, 3

# read data files

In [3]:
df_raw = pd.read_csv('raw_data/raw_study_3.csv')

C:\Users\Cinti\AppData\Local\Temp\ipykernel_7212\3630403981.py:1: DtypeWarning: Columns (2,3,7,11,15,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('C:/Users/Cinti/Documents/Kutatás/AUTI/_to_publish/raw_data/raw_study_3.csv')


# preprocessing

#### filter unnecessary columns

In [4]:
if study_number == 1 or study_number == 2:
    df = df_raw.loc[:, ['Group', 'Subject', 'Block', 'Epoch', 'finalRT', 'ACC', 'TrialType', 'triplet_type', 'trial_number']]
elif study_number == 3:
    df = df_raw.loc[:, ['Subject', 'Block', 'Epoch', 'finalRT', 'ACC', 'triplet_type', 'trial_number', 'is_practice', 'first_response']]

#### drop participants

In [10]:
if study_number == 1:
    participants_to_drop = [4, 16, 20, 21, 22, 23, 24, 25, 108]
elif study_number == 2:
    participants_to_drop = [2, 7, 28]
elif study_number == 3:
    participants_to_drop = ['zfbr1xmr', 'ewa5a6u1', 'oah1o7vp', 'lc8zqd2x', 'i2dl9e3g', 'ln7eke9h', '3ippg8nb', 'qzmrvkrj', 'dzlz88d0', 'humixa7l', 'j21b81w9', 'ogw7lvdc', '0fp60qq4', 'n80gbsvq', 'i07caajj', 'isfjzkx5', '9l7yus87', 'wmltviex', 'j21b81w9', 'ogw7lvdc', '0fp60qq4', 'n80gbsvq', 'i07caajj', 'isfjzkx5', '9l7yus87', 'wmltviex', '1x978q7u', '2k3uo9o4', 'b8xtm1zh', 'f0akmxef', 'gt3prp03', 'h6qpfwbn', 'ls9k0t3b', 'nu06la2p', 'orjkqq2x', 'zbizxozj', '5rkxbz6t', 'arq89iuv', 'jiu3yo02', 'vjyzdao1', 'wokamkap', '1r2rgkqt', '1jgexvhk', 'u81zondt', 's1v4un3e', 'xvipcks4', 'xbiiwl18', 'smu50mjz', '8dcnhu5e', 'e550q34s', 'mfzonm1o', 'y25hszee', 'p7juo6b2', 'kd1s2st3', '89bnh3gg', 'hqqp11vo', '3yl5creu', '73fy2ybt', '4h9lbkef', 'gh8haajx', 'wxsg8bsv', 'tozvyp0y', 'ag4evd41', 'hbo6llus', 'hejp7uzx', 'qry7e8zc']

In [11]:
df = df[~df['Subject'].isin(participants_to_drop)].reset_index(inplace=False)

#### split based on median AQ points

In [ ]:
if study_number == 3:
    # load dataframe with the AQ points
    AQ_df = pd.read_excel('raw_data/aq_recoded.xlsx')
    AQ_df = AQ_df[['Subject', 'AQ_total']]
    
    # merge with dataframe
    df = df.merge(AQ_df, on = 'Subject', how = 'left')
    
    # split based on the median AQ
    AQ_median = df['AQ_total'].median()
    df['Group'] = df['AQ_total'].apply(lambda x: 0 if x < AQ_median else 1)

#### drop the practice trials and blocks

In [ ]:
if study_number == 1 or study_number == 2:
    indexNames = df[df['TrialType'] == 'Prac'].index
    df.drop(indexNames, inplace=True)
    df = df.reset_index(drop=True)
elif study_number == 3:
    indexNames = df[df['is_practice'] == 1].index
    df.drop(indexNames, inplace=True)
    df = df.reset_index(drop=True)

#### drop the first two trials

In [ ]:
indexNames = df[df['triplet_type'] == 'X'].index
df.drop(indexNames, inplace=True)

#### drop trials that were not first responses to a given stimulus

In [ ]:
if study_number == 3:
    indexNames1 = df[df['first_response'] == 0].index
    df.drop(indexNames1, inplace=True)
    df = df.reset_index(drop=True)

#### creating the bins

In [ ]:
if study_number == 1 or study_number == 2:
    df.loc[df['trial_number'] <= 21, 'bin'] = 1
    df.loc[(df['trial_number'] > 21) & (df['trial_number'] <= 37), 'bin'] = 2
    df.loc[(df['trial_number'] > 37) & (df['trial_number'] <= 53), 'bin'] = 3
    df.loc[(df['trial_number'] > 53) & (df['trial_number'] <= 69), 'bin'] = 4
    df.loc[(df['trial_number'] > 69) & (df['trial_number'] <= 85), 'bin'] = 5  
elif study_number == 3:
    df.loc[df['trial_number'] <= 16, 'bin'] = 1
    df.loc[(df['trial_number'] > 16) & (df['trial_number'] <= 32), 'bin'] = 2
    df.loc[(df['trial_number'] > 32) & (df['trial_number'] <= 48), 'bin'] = 3
    df.loc[(df['trial_number'] > 48) & (df['trial_number'] <= 64), 'bin'] = 4
    df.loc[(df['trial_number'] > 64) & (df['trial_number'] <= 80), 'bin'] = 5

#### exclude, trills and repetitions

In [ ]:
indexNames1 = df[df['triplet_type'] == 'T'].index
df.drop(indexNames1, inplace=True)
indexNames2 = df[df['triplet_type'] == 'R'].index
df.drop(indexNames2, inplace=True)
df = df.reset_index(drop=True)

#### define outliers based on boxplot

In [ ]:
def get_outliers(data, value):
    # define outlier boundaries (for each subject, for each epoch)
    data['outlier'] = 0
    n_epoch = data['Epoch'].nunique()
    n_group = data['Group'].nunique()
    
    for g in range(n_group):
        subjects = data[data['Group'] == g]['Subject'].unique()
        n_subject = data[data['Group'] == g]['Subject'].nunique()
        
        for s in range(n_subject):
            q1 = data[data['Group'] == g][data['Subject'] == subjects[s]].groupby('Epoch')[value].quantile(q = 0.25).to_numpy()
            q3 = data[data['Group'] == g][data['Subject'] == subjects[s]].groupby('Epoch')[value].quantile(q = 0.75).to_numpy()
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            
            # create a new variale indicating if the trial is an outlier
            for i in range(len(data)):
                if data['Group'][i] == g:
                    if data['Subject'][i] == subjects[s]:
                        for e in range(n_epoch):
                            if data['Epoch'][i] == e + 1:
                                if (data[value][i] < lower[e]) | (data[value][i] > upper[e]):
                                    data['outlier'][i] = 1
                                else:
                                    data['outlier'][i] = 0
    return data          

In [ ]:
df = get_outliers(df, 'finalRT')

In [ ]:
indexNames0 = df[df['outlier'] == 1].index
df.drop(indexNames0, inplace=True)
df = df.reset_index(drop=True)

#### drop incorrect trials

In [ ]:
indexNames = df[df['ACC'] == 0].index
df.drop(indexNames, inplace=True)
df = df.reset_index(drop=True)

# Calculating offline-online learning in statistical learning

#### Calculate H and L scores by bins

In [ ]:
SL_RT_wide = df.copy()

if study_number == 1 or study_number == 2:
    SL_RT_wide= SL_RT_wide.pivot_table(index=['Group','Subject'], columns=['Block','bin','triplet_type'], values='finalRT', aggfunc=np.mean)
    SL_RT_wide = SL_RT_wide.rename(columns={1.0: 'block1', 2.0: 'block2', 3.0:'block3', 4.0:'block4', 5.0: 'block5', 6.0: 'block6', 7.0: 'block7', 8.0:'block8', 9.0:'block9', 10.0: 'block10',
                                        11.0: 'block11', 12.0: 'block12', 13.0:'block13', 14.0:'block14', 15.0: 'block15', 16.0: 'block16', 17.0: 'block17', 18.0:'block18', 19.0:'block19', 20.0: 'block20'}, level=0)
    SL_RT_wide = SL_RT_wide.rename(columns={1.0: 'bin1', 2.0: 'bin2', 3.0:'bin3', 4.0:'bin4', 5.0: 'bin5'}, level=1)

    SL_RT_wide.columns = SL_RT_wide.columns.map('_'.join).str.strip('_')
elif study_number == 3:
    SL_RT_wide= SL_RT_wide.pivot_table(index=['Group','AQ_total', 'Subject'], columns=['Block','bin','triplet_type'], values='finalRT', aggfunc=np.mean)
    SL_RT_wide = SL_RT_wide.rename(columns={1.0: 'block1', 2.0: 'block2', 3.0:'block3', 4.0:'block4', 5.0: 'block5', 6.0: 'block6', 7.0: 'block7', 8.0:'block8', 9.0:'block9', 10.0: 'block10',
                                            11.0: 'block11', 12.0: 'block12', 13.0:'block13', 14.0:'block14', 15.0: 'block15', 16.0: 'block16', 17.0: 'block17', 18.0:'block18', 19.0:'block19', 20.0: 'block20',
                                            21.0: 'block21', 22.0: 'block22', 23.0:'block23', 24.0:'block24', 25.0: 'block25'}, level=0)
    SL_RT_wide = SL_RT_wide.rename(columns={1.0: 'bin1', 2.0: 'bin2', 3.0:'bin3', 4.0:'bin4', 5.0: 'bin5'}, level=1)
    
    SL_RT_wide.columns = SL_RT_wide.columns.map('_'.join).str.strip('_')

SL_RT_wide.head()

#### Calculate offline and online learning scores by bins

In [ ]:
if study_number == 1 or study_number == 2:
    #calculate learning scores for bin 1 and 5
    for x in range(1,21):
        for y in [1,5]:
            SL_RT_wide['block'+str(x)+'_bin'+str(y)+'_SL'] = SL_RT_wide['block'+str(x)+'_bin'+str(y)+'_L'] - SL_RT_wide['block'+str(x)+'_bin'+str(y)+'_H']
    
    #calculate offline learning scores
    for x in range(2,21):
        SL_RT_wide['block'+str(x)+'_offline'] = SL_RT_wide['block'+str(x)+'_bin1_SL'] - SL_RT_wide['block'+str(x-1)+'_bin5_SL']
    
    #calculate online learning scores
    for x in range(1,21):
        SL_RT_wide['block'+str(x)+'_online'] = SL_RT_wide['block'+str(x)+'_bin5_SL'] - SL_RT_wide['block'+str(x)+'_bin1_SL']

elif study_number == 3:
    #calculate learning scores for bin 1 and 5
    for x in range(1,26):
        for y in [1,5]:
            SL_RT_wide['block'+str(x)+'_bin'+str(y)+'_SL'] = SL_RT_wide['block'+str(x)+'_bin'+str(y)+'_L'] - SL_RT_wide['block'+str(x)+'_bin'+str(y)+'_H']
    
    #calculate offline learning scores
    for x in range(2,26):
        SL_RT_wide['block'+str(x)+'_offline'] = SL_RT_wide['block'+str(x)+'_bin1_SL'] - SL_RT_wide['block'+str(x-1)+'_bin5_SL']
    
    #calculate online learning scores
    for x in range(1,26):
        SL_RT_wide['block'+str(x)+'_online'] = SL_RT_wide['block'+str(x)+'_bin5_SL'] - SL_RT_wide['block'+str(x)+'_bin1_SL']
    
SL_RT_wide.head()

#### Calculate overall offline and online learning scores

In [ ]:
if study_number == 1 or study_number == 2:
    SL_RT_wide['offline_SL'] = SL_RT_wide[['block2_offline','block3_offline', 'block4_offline','block5_offline', 'block6_offline','block7_offline', 'block8_offline','block9_offline', 'block10_offline',
                                           'block11_offline', 'block12_offline','block13_offline', 'block14_offline','block15_offline', 'block16_offline', 'block17_offline','block18_offline', 'block19_offline','block20_offline']].mean(axis=1)
                                           
    SL_RT_wide['online_SL'] = SL_RT_wide[['block1_online', 'block2_online','block3_online', 'block4_online','block5_online', 'block6_online','block7_online', 'block8_online','block9_online', 'block10_online',
                                           'block11_online', 'block12_online','block13_online', 'block14_online','block15_online', 'block16_online', 'block17_online','block18_online', 'block19_online','block20_online']].mean(axis=1)
    
elif study_number == 3:
    SL_RT_wide['offline_SL'] = SL_RT_wide[['block2_offline','block3_offline', 'block4_offline','block5_offline', 'block6_offline','block7_offline', 'block8_offline','block9_offline', 'block10_offline',
                                           'block11_offline', 'block12_offline','block13_offline', 'block14_offline','block15_offline', 'block16_offline', 'block17_offline','block18_offline', 'block19_offline','block20_offline',
                                           'block21_offline', 'block22_offline','block23_offline', 'block24_offline', 'block25_offline']].mean(axis=1)
                                           
    SL_RT_wide['online_SL'] = SL_RT_wide[['block1_online', 'block2_online','block3_online', 'block4_online','block5_online', 'block6_online','block7_online', 'block8_online','block9_online', 'block10_online',
                                           'block11_online', 'block12_online','block13_online', 'block14_online','block15_online', 'block16_online', 'block17_online','block18_online', 'block19_online','block20_online',
                                           'block21_online', 'block22_online','block23_online', 'block24_online','block25_online']].mean(axis=1)
    
SL_RT_wide.head()

#### add new columns indicating if they learned offline/online

In [ ]:
SL_RT_wide['is_learn_offline'] = SL_RT_wide['offline_SL'].apply(lambda x: 1 if x > 0 else 0)
SL_RT_wide['is_learn_online'] = SL_RT_wide['online_SL'].apply(lambda x: 1 if x > 0 else 0)
SL_RT_wide.head()

#### Save data in CSV

In [ ]:
SL_RT_wide.to_csv('processed_data/study_3/data_study_3_SL_offline_online.csv', index=True)

# Calculating offline-online learning in general skill learning

#### Calculate general RT scores by bins

In [ ]:
GS_RT_wide = df.copy()

if study_number == 1 or study_number == 2:
    GS_RT_wide= GS_RT_wide.pivot_table(index=['Group', 'Subject'], columns=['Block','bin'], values='finalRT', aggfunc=np.mean)
    GS_RT_wide = GS_RT_wide.rename(columns={1.0: 'block1', 2.0: 'block2', 3.0:'block3', 4.0:'block4', 5.0: 'block5',6.0: 'block6', 7.0: 'block7', 8.0:'block8', 9.0:'block9', 10.0: 'block10',
                                            11.0: 'block11', 12.0: 'block12', 13.0:'block13', 14.0:'block14', 15.0: 'block15', 16.0: 'block16', 17.0: 'block17', 18.0:'block18', 19.0:'block19', 20.0: 'block20'}, level=0)
    GS_RT_wide = GS_RT_wide.rename(columns={1.0: 'bin1', 2.0: 'bin2', 3.0:'bin3', 4.0:'bin4', 5.0: 'bin5'}, level=1)
    
    GS_RT_wide.columns = GS_RT_wide.columns.map('_'.join).str.strip('_')
    
elif study_number == 3:
    GS_RT_wide= GS_RT_wide.pivot_table(index=['Group','AQ_total', 'Subject'], columns=['Block','bin'], values='finalRT', aggfunc=np.mean)
    GS_RT_wide = GS_RT_wide.rename(columns={1.0: 'block1', 2.0: 'block2', 3.0:'block3', 4.0:'block4', 5.0: 'block5',6.0: 'block6', 7.0: 'block7', 8.0:'block8', 9.0:'block9', 10.0: 'block10',
                                            11.0: 'block11', 12.0: 'block12', 13.0:'block13', 14.0:'block14', 15.0: 'block15', 16.0: 'block16', 17.0: 'block17', 18.0:'block18', 19.0:'block19', 20.0: 'block20',
                                            21.0: 'block21', 22.0: 'block22', 23.0:'block23', 24.0:'block24', 25.0: 'block25'}, level=0)
    GS_RT_wide = GS_RT_wide.rename(columns={1.0: 'bin1', 2.0: 'bin2', 3.0:'bin3', 4.0:'bin4', 5.0: 'bin5'}, level=1)
    
    GS_RT_wide.columns = GS_RT_wide.columns.map('_'.join).str.strip('_')
    
GS_RT_wide.head()

#### Calculate offline and online learning scores by bins

In [ ]:
if study_number == 1 or study_number == 2:
    #calculate offline learning scores
    for x in range(2,21):
        GS_RT_wide['block'+str(x)+'_offline'] = GS_RT_wide['block'+str(x)+'_bin1'] - GS_RT_wide['block'+str(x-1)+'_bin5']
    
    #calculate online learning scores
    for x in range(1,21):
        GS_RT_wide['block'+str(x)+'_online'] = GS_RT_wide['block'+str(x)+'_bin5'] - GS_RT_wide['block'+str(x)+'_bin1']

elif study_number == 3:
    #calculate offline learning scores
    for x in range(2,26):
        GS_RT_wide['block'+str(x)+'_offline'] = GS_RT_wide['block'+str(x)+'_bin1'] - GS_RT_wide['block'+str(x-1)+'_bin5']
    
    #calculate online learning scores
    for x in range(1,26):
        GS_RT_wide['block'+str(x)+'_online'] = GS_RT_wide['block'+str(x)+'_bin5'] - GS_RT_wide['block'+str(x)+'_bin1']
        
GS_RT_wide.head()

#### Calculate overall offline and online learning scores

In [ ]:
if study_number == 1 or study_number == 2:
    GS_RT_wide['offline_GS'] = GS_RT_wide[['block2_offline','block3_offline', 'block4_offline','block5_offline', 'block6_offline','block7_offline', 'block8_offline','block9_offline', 'block10_offline',
                                           'block11_offline', 'block12_offline','block13_offline', 'block14_offline','block15_offline', 'block16_offline', 'block17_offline','block18_offline', 'block19_offline','block20_offline']].mean(axis=1)
                                           
    GS_RT_wide['online_GS'] = GS_RT_wide[['block1_online', 'block2_online','block3_online', 'block4_online','block5_online', 'block6_online','block7_online', 'block8_online','block9_online', 'block10_online',
                                           'block11_online', 'block12_online','block13_online', 'block14_online','block15_online', 'block16_online', 'block17_online','block18_online', 'block19_online','block20_online']].mean(axis=1)
elif study_number == 3:
    GS_RT_wide['offline_GS'] = GS_RT_wide[['block2_offline','block3_offline', 'block4_offline','block5_offline', 'block6_offline','block7_offline', 'block8_offline','block9_offline', 'block10_offline',
                                           'block11_offline', 'block12_offline','block13_offline', 'block14_offline','block15_offline', 'block16_offline', 'block17_offline','block18_offline', 'block19_offline','block20_offline',
                                           'block21_offline', 'block22_offline','block23_offline', 'block24_offline', 'block25_offline']].mean(axis=1)
                                           
    GS_RT_wide['online_GS'] = GS_RT_wide[['block1_online', 'block2_online','block3_online', 'block4_online','block5_online', 'block6_online','block7_online', 'block8_online','block9_online', 'block10_online',
                                           'block11_online', 'block12_online','block13_online', 'block14_online','block15_online', 'block16_online', 'block17_online','block18_online', 'block19_online','block20_online',
                                           'block21_online', 'block22_online','block23_online', 'block24_online','block25_online']].mean(axis=1)

GS_RT_wide.head()

In [ ]:
GS_RT_wide['is_learn_offline'] = GS_RT_wide['offline_GS'].apply(lambda x: 0 if x > 0 else 1)
GS_RT_wide['is_learn_online'] = GS_RT_wide['online_GS'].apply(lambda x: 0 if x > 0 else 1)
GS_RT_wide.head()

#### Save the data to CSV

In [ ]:
GS_RT_wide.to_csv('processed_data/study_3/data_study_3_GS_offline_online.csv',index=True)